# Calculating derived outputs

In the [previous notebook](/textbook/01-basic-model.html), we saw how to create and run a simple compartmental model.

This notebook builds on this to show how to request more detailed outputs from the model, in addition to just the compartment sizes. _summer_ supports the calculation of "derived outputs" - these are additional outputs that can be calculated from either:

- The model compartment sizes for each timestep; or
- The model flow rates at each timestep

In epidemiology, we often want to estimate outputs from our models that are not just the absolute size of the individual explicitly modelled compartments or states.
These quantities are not compartment sizes, but rather the absolute (not per capita) rate of transition between one state and another.
Although such quantities may be part of the calculation that is used to update the population distribution from one time step to the next,
these transition rates are not generally tracked during the numeric integration process.
In epidemiology, one of the clearest examples of the need for this is the need to understand "incident" as well as "prevalent" quantities.
Compartment sizes or the ratios of compartment sizes to the total population or to other quantities only provide us with information on the prevalence
or relative prevalence of certain states within the overall population,
and even these prevalent quantities may be more complicated than just the total number of people in a single compartment.

To extend the range of outputs that can be estimated from a _summer_ model object, there are several different types of derived outputs that are supported:

- **[Flow output](/api/model.html#summer.model.CompartmentalModel.request_output_for_flow)**: A sum of one or more flow rates at each timestep
- **[Compartment output](/api/model.html#summer.model.CompartmentalModel.request_output_for_compartments)**: A sum of one or more compartment sizes at each timestep
- **[Aggregate output](/api/model.html#summer.model.CompartmentalModel.request_aggregate_output)**: An aggregate of other derived outputs
- **[Cumulative output](/api/model.html#summer.model.CompartmentalModel.request_cumulative_output)**: A cumulative sum of another derived output
- **[Function output](/api/model.html#summer.model.CompartmentalModel.request_function_output)**: A pure function of other derived outputs

As usual, let's start with our import statements.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from summer import CompartmentalModel

pd.options.plotting.backend = "plotly"

Then let's generate a simple model, exactly was we did in the previous notebook.

In [ ]:
def get_base_sir_model():
    """
    Generate an instance of an SIR model with some fixed parameters, 
    population distribution and parameters.
    
    Returns:
        model: The SIR compartmental model
    """
    compartments = (
        "susceptible",
        "infectious",
        "recovered",
    )
    analysis_times = (0, 20)
    
    model = CompartmentalModel(
        times=analysis_times,
        compartments=compartments,
        infectious_compartments=["infectious"],
    )
    model.set_initial_population(
        distribution={"susceptible": 990, "infectious": 10}
    )
    model.add_infection_frequency_flow(
        name="infection", 
        contact_rate=1.,
        source="susceptible", 
        dest="infectious",
    )
    model.add_transition_flow(
        name="recovery", 
        fractional_rate=0.333,
        source="infectious", 
        dest="recovered",
    )
    model.add_death_flow(
        name="infection_death", 
        death_rate=0.05,
        source="infectious",
    )
    return model

As previously, this can easily be run and the compartment sizes plotted.

In [ ]:
sir_model = get_base_sir_model()
sir_model.run()
compartment_values = sir_model.get_outputs_df()
compartment_values.plot()

Next let's look at the various types of derived outputs supported by _summer_.

## Compartment outputs

A **[compartment output](/model.html#summer.model.CompartmentalModel.request_output_for_compartments)** tracks the sum of one or more compartments at each timestep. These requests can also select compartments for particular strata in a stratified model, as demonstrated in later notebooks.

In [ ]:
base_model = get_base_sir_model()

# Request that the infectious and recovered compartment sizes are combined into 'ever_infected'
base_model.request_output_for_compartments(
    name="ever_infected", 
    compartments=["infectious", "recovered"]
)

base_model.run()
derived_outputs = base_model.get_derived_outputs_df()
derived_outputs.plot()

## Flow outputs

A **[flow output](/api/model.html#summer.model.CompartmentalModel.request_output_for_flow)** tracks a set of requested flow rates for each timestep. These requests can also select flows between particular strata in a stratified model (see later examples).
For example, we might want to ask the model to track the number of people who died from infection per timestep. Note that this is not represented by any of the explicitly modelled states.

In [ ]:
sir_model = get_base_sir_model()

# Request that the model calculate a derived output when it is run
sir_model.request_output_for_flow(
    name="deaths", 
    flow_name="infection_death"
)

Now when we run the model, we can obtain a pandas dataframe representing the new infection-related daily deaths that can be accessed through `model.get_derived_outputs_df`, as follows.

In [ ]:
# Run the model
sir_model.run()

# View the derived outputs that were calculated when the `run()` method was called
modelled_deaths = sir_model.get_derived_outputs_df()
modelled_deaths.plot()

### Distinguishing incidence from infection
Let's build a new model that incorporates an explicit delay between infection and later progression or activation to the infectious compartment.
Because we still only allow that the compartment called "_infectious_" is actually infectious (contributes to the force of infection calculation),
we therefore have a delay between the process of being infected by someone else and progressing to become infectious yourself.

In [ ]:
def get_base_seir_model():
    """
    An adaptation of the SIR model introduced above, with a couple of
    small differences to turn it into an SEIR model.
    Generate an instance of an SEIR model with some fixed parameters, 
    population distribution and parameters.
    
    Returns:
        model: The SEIR compartmental model
    """
    compartments = (
        "susceptible",
        "exposed",
        "infectious",
        "recovered",
    )
    analysis_times = (0, 20)
    
    model = CompartmentalModel(
        times=analysis_times,
        compartments=compartments,
        infectious_compartments=["infectious"],
    )
    model.set_initial_population(
        distribution={"susceptible": 990, "infectious": 10}
    )
    model.add_infection_frequency_flow(
        name="infection", 
        contact_rate=1.,
        source="susceptible", 
        dest="exposed",  # This is different from the SIR model
    )
    # This flow didn't exist in the SIR model
    model.add_transition_flow(
        name="progression",
        fractional_rate=0.333,
        source="exposed",
        dest="infectious",
    )
    model.add_transition_flow(
        name="recovery", 
        fractional_rate=0.333,
        source="infectious", 
        dest="recovered",
    )
    model.add_death_flow(
        name="infection_death", 
        death_rate=0.05,
        source="infectious",
    )
    return model

Let's track these transitions (infection and progression) explicitly.
The process of infection is intuitive, but let's refer to the process of progressing from the 
exposed to the infectious compartment as "incidence",
because it represents the rate at which new disease episodes occur.

In [ ]:
seir_model = get_base_seir_model()

seir_model.request_output_for_flow(
    name="infection",
    flow_name="infection",
)
seir_model.request_output_for_flow(
    name="incidence", 
    flow_name="progression",
)

Looks like these quantities are pretty similar, but not identical, in this model.
This is consistent with what we would expect.
The delay from individuals being infected to progressing to active disease
is directly reflected in the delay in the changes in the infection and incidence quantities.

In [ ]:
seir_model.run()
derived_outputs = seir_model.get_derived_outputs_df()
derived_outputs.plot()

## Cumulative outputs

You can use a  **[cumulative output](/model.html#summer.model.CompartmentalModel.request_cumulative_output)** to request that the model tracks the cumulative sum of other derived outputs over time. For example, let's track total infection deaths and the total people recovered:

In [ ]:
model = build_base_sir_model()
model.request_output_for_flow(name="deaths", flow_name="infection_death")

# Request that the 'deaths' derived output is accumulated into 'deaths_cumulative'.
model.request_cumulative_output(name="deaths_cumulative", source="deaths")

model.run()
plot_output(model, "deaths_cumulative", "Cumulative infection deaths")

## Aggregate outputs

You can use an **[aggregate output](/model.html#summer.model.CompartmentalModel.request_aggregate_output)** to request an aggregate of other derived outputs.

In [ ]:
model = build_model()

# Track some flows.
model.request_output_for_flow(name="deaths", flow_name="infection_death")
model.request_output_for_flow(name="recoveries", flow_name="recovery")

# Accumulate the flows.
model.request_cumulative_output(name="deaths_cumulative", source="deaths")
model.request_cumulative_output(name="recoveries_cumulative", source="recoveries")

# Aggregate 'deaths_cumulative' and 'recovered_cumulative' into a single output.
model.request_aggregate_output(
    name="dead_or_recovered_cumulative",
    sources=["deaths_cumulative", "recoveries_cumulative"]
)

model.run()
plot_output(model, "dead_or_recovered_cumulative", "Cumulative dead or recovered")
# (In this simple model, this could be also easily be tracked as the complement of the the susceptible population.)

## Function outputs

You can use **[function outputs](/model.html#summer.model.CompartmentalModel.request_function_output)** to calculate outputs with a user-defined function. This function takes the requested sources (NumPy arrays) as inputs and should return a NumPy array of the same size.

For example, here we request a calculation that gets us the prevalence of the disease.

In [ ]:
model = build_model()

# Track the number of infectious people as a derived output.
# Here we use `save_results=False` because we don't need to use this later.
model.request_output_for_compartments(name="count_infectious", compartments=["I"], save_results=False)

# Track the total population as a derived output.
model.request_output_for_compartments(name="total_population", compartments=["S", "R"], save_results=False)

# Define a function that we will use to calculate prevalence.
# The arguments (infectious, total) will be NumPy arrays.
def get_prevalence(infectious, total):
    return infectious / total

# Request a function output, using `get_prevalence`.
# The sources will map to the function arguments.
model.request_function_output(
    name="prevalence",
    sources=["count_infectious", "total_population"],
    func=get_prevalence,
)

model.run()
plot_output(model, "prevalence", "Prevalence of disease")

## Summary

That's it for now, now you know how to:

- Request derived outputs
- Chain and combine derived outputs
- Access and visualize the derived outputs

A detailed API reference of the CompartmentalModel class can be found [here](http://summerepi.com/api/model.html)